In [ ]:
import pandas as pd
users = pd.read_csv(
    "ml-1m/users.dat",
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
    engine='python'
)

movies = pd.read_csv(
    "ml-1m/movies.dat", sep="::", names=["movie_id", "title", "genres"],
    engine='python',encoding='ISO-8859-1'
)

ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    engine='python'
)


In [20]:
print("Users data:")
print(users.head())
print("\nMovies data:")
print(movies.head())
print("\nRatings data:")
print(ratings.head())

Users data:
   user_id sex  age_group  occupation zip_code
0        1   F          1          10    48067
1        2   M         56          16    70072
2        3   M         25          15    55117
3        4   M         45           7    02460
4        5   M         25          20    55455

Movies data:
   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy

Ratings data:
   user_id  movie_id  rating  unix_timestamp
0        1      1193       5       978300760
1        1       661       3       978302109
2        1       914       3       978301968
3        1      3408       4 

In [21]:
# 将原始ID转换为连续的索引，便于embedding层使用
def generate_remap_id_dict(df,col):
    ids = df[df[col].notnull()][col].unique().tolist()
    ids = sorted(ids)
    id_map_dict = {x: i+1 for i, x in enumerate(ids)}
    id_map_dict["UNK"]=0
    df[f"{col}_index"] = df[col].fillna("UNK").map(id_map_dict)
    return id_map_dict


In [22]:
user_id_map_dict=generate_remap_id_dict(users,col='user_id')
user_sex_map_dict=generate_remap_id_dict(users,col='sex')
user_age_group_map_dict=generate_remap_id_dict(users,col='age_group')
user_occupation_map_dict=generate_remap_id_dict(users,col='occupation')
movie_id_map_dict = generate_remap_id_dict(movies,col='movie_id')


In [23]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
ratings["norm_rating"] = min_max_scaler.fit_transform(
    ratings["rating"].values.reshape(-1, 1))[:, 0]

In [24]:
users.head()

,user_id,sex,age_group,occupation,zip_code,user_id_index,sex_index,age_group_index,occupation_index
0,1,F,1,10,48067,1,1,1,11
1,2,M,56,16,70072,2,2,7,17
2,3,M,25,15,55117,3,2,3,16
3,4,M,45,7,02460,4,2,5,8
4,5,M,25,20,55455,5,2,3,21


In [25]:
df_user_full_matrix = users.merge(ratings[['user_id', 'movie_id', 'norm_rating','unix_timestamp']], on='user_id', how='left')
df_user_full_matrix['movie_id_index'] = ratings['movie_id'].map(movie_id_map_dict)
df_user_full_matrix['user_id_index'] = ratings['user_id'].map(user_id_map_dict)
df_user_full_matrix['sex_index'] = df_user_full_matrix['sex'].map(user_sex_map_dict)
df_user_full_matrix['age_group_index'] = df_user_full_matrix['age_group'].map(user_age_group_map_dict)
df_user_full_matrix['occupation_index'] = df_user_full_matrix['occupation'].map(user_occupation_map_dict)
df_user_full_matrix = df_user_full_matrix[['user_id_index', 'sex_index', 'age_group_index', 'occupation_index', 'movie_id_index', 'norm_rating','unix_timestamp']]
df_user_full_matrix.head()

,user_id_index,sex_index,age_group_index,occupation_index,movie_id_index,norm_rating,unix_timestamp
0,1,1,1,11,1177,1.00,978300760
1,1,1,1,11,656,0.50,978302109
2,1,1,1,11,903,0.50,978301968
3,1,1,1,11,3340,0.75,978300275
4,1,1,1,11,2287,1.00,978824291


In [26]:
# sort with user_id and unix_timestamp
df_user_full_matrix = df_user_full_matrix.sort_values(['user_id_index', 'unix_timestamp'])
df_user_full_matrix.head()

,user_id_index,sex_index,age_group_index,occupation_index,movie_id_index,norm_rating,unix_timestamp
31,1,1,1,11,3118,0.75,978300019
22,1,1,1,11,1251,1.00,978300055
27,1,1,1,11,1673,0.75,978300055
37,1,1,1,11,1010,1.00,978300055
24,1,1,1,11,2272,0.50,978300103


In [27]:
def gen_sequence_data(df, window_size, step):
    sequences = []
    
    for user_id, user_data in df.groupby('user_id_index'):
        user_data = user_data.reset_index(drop=True)
    
        for i in range(0, len(user_data) - window_size + 1, step):
            sequence = user_data.iloc[i:i + window_size]
            
            movie_sequence = sequence['movie_id_index'].tolist()
            rating_sequence = sequence['norm_rating'].tolist()
            
            sequences.append({
                'user_id_index': user_id,
                'movie_sequence': movie_sequence,
                'rating_sequence': rating_sequence,
                'sex_index': sequence['sex_index'].iloc[0],
                'age_group_index': sequence['age_group_index'].iloc[0],
                'occupation_index': sequence['occupation_index'].iloc[0]
            })
    
    return pd.DataFrame(sequences)
df_user_view = gen_sequence_data(df_user_full_matrix,window_size=4,step=2)
df_user_view.head()

,user_id_index,movie_sequence,rating_sequence,sex_index,age_group_index,occupation_index
0,1,"[3118, 1251, 1673, 1010]","[0.75, 1.0, 0.75, 1.0]",1,1,11
1,1,"[1673, 1010, 2272, 1769]","[0.75, 1.0, 0.5, 1.0]",1,1,11
2,1,"[2272, 1769, 3340, 2736]","[0.5, 1.0, 0.75, 1.0]",1,1,11
3,1,"[3340, 2736, 1190, 1177]","[0.75, 1.0, 0.75, 1.0]",1,1,11
4,1,"[1190, 1177, 712, 258]","[0.75, 1.0, 0.5, 0.75]",1,1,11


In [28]:
import numpy as np
random_selection = np.random.rand(len(df_user_view)) <= 0.85
train_data = df_user_view[random_selection]
test_data = df_user_view[~random_selection]
print(len(train_data),len(test_data))

419142 73441


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Dict, List, Optional, Union
import math

class EmbeddingLayer(nn.Module):
    """
    BST模型的统一Embedding层
    """
    def __init__(self, 
                 embed_configs: Dict[str, Dict],
                 dropout: float = 0.2,
                 initialization: str = "xavier"):
        super().__init__()
        
        self.embed_configs = embed_configs
        self.dropout = dropout
        self.embed_dim = embed_configs['position']['embed_dim']
        self.seq_len = embed_configs['position']['num_embed']
        
        
        # 创建embedding层
        self.embeddings = nn.ModuleDict() # 各特征的Embedding
        self.feature_types = {}
        
        for feature_name, config in embed_configs.items():
            embed_dim_feat = config.get('embed_dim',self.embed_dim)
            num_embeddings = config['num_embed']
            feature_type = config.get('type', 'categorical')
            
            # 根据特征类型创建不同的embedding
            if feature_type == 'categorical':
                self.embeddings[feature_name] = nn.Embedding(
                    num_embeddings, embed_dim_feat, padding_idx=0
                )
            elif feature_type == 'sequence':
                # 序列特征
                self.embeddings[feature_name] = SequenceEmbedding(
                    num_embeddings, embed_dim_feat
                )
            
            self.feature_types[feature_name] = feature_type
            
            # 初始化embedding权重
            self._init_embedding(self.embeddings[feature_name], initialization)
            # inner
        # outer
        # Dropout层
        self.embedding_dropout = nn.Dropout(dropout)
        
        # cal dims
        total_dim = 0
        transformer_dim = 0
        for k,v in embed_configs.items():
            total_dim += v["embed_dim"]
            if k == 'item' or k =='position':
                transformer_dim += v["embed_dim"]
        total_dim += embed_configs['item']['embed_dim']
        self.total_dim = total_dim
        self.transformer_dim = transformer_dim
        
        
    def _init_embedding(self, embedding_layer, init_type):
        """初始化embedding权重"""
        if hasattr(embedding_layer, 'weight'):
            if init_type == "xavier":
                nn.init.xavier_uniform_(embedding_layer.weight)
            elif init_type == "normal":
                nn.init.normal_(embedding_layer.weight, std=0.1)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(embedding_layer.weight)
    
    def forward(self, features: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        embeddings = {}
        
        for feature_name, feature_tensor in features.items():
            if feature_name in self.embeddings:
                # 获取embedding
                embed = self.embeddings[feature_name](feature_tensor)
                # 应用dropout
                embed = self.embedding_dropout(embed)
                embeddings[feature_name] = embed
        return embeddings
    
class SequenceEmbedding(nn.Module):
    def __init__(self, num_item, embed_dim,seq_length):
        super().__init__()
        self.item_embedding = nn.Embedding(num_item, embed_dim, padding_idx=0)
        
        self.position_embedding = nn.Embedding(seq_length, embed_dim)
        
        self.layer_norm = nn.LayerNorm(embed_dim * 2)  # 拼接
        
    def forward(self, movie_sequence, rating_sequence=None):
        batch_size, seq_len = movie_sequence.shape
        
        item_embeds = self.item_embedding(movie_sequence)  # (B, L, D)
        
        # Position embeddings
        positions = torch.arange(seq_len, device=movie_sequence.device)
        pos_embeds = self.position_embedding(positions).unsqueeze(0).expand(batch_size, -1, -1)
        
        # concat
        sequence_embeds = torch.cat([item_embeds, pos_embeds], dim=-1)  # (B, L, 2*D)
        
        # Optional: Rating weighting (BST uses ratings as attention weights)
        if rating_sequence is not None:
            rating_weights = rating_sequence.unsqueeze(-1)  # (B, L, 1)
            sequence_embeds = sequence_embeds * rating_weights
            
        return self.layer_norm(sequence_embeds)



In [ ]:
# embedding config

num_user = len(user_id_map_dict)
num_movie = len(movie_id_map_dict)
num_occupation = len(user_occupation_map_dict)
num_age_group = len(user_age_group_map_dict)
num_sex = len(user_sex_map_dict)

embed_configs = {}
EMED_DIM=32
SEQUENCE_SIZE = 4
embed_configs['item']={"embed_dim":EMED_DIM,"num_embed":num_movie}
embed_configs['position'] = {"embed_dim":EMED_DIM,"num_embed":SEQUENCE_SIZE}

embed_configs['user']={"embed_dim":EMED_DIM,"num_embed":num_user}
embed_configs['sex'] = {"embed_dim": EMED_DIM, "num_embed":num_sex }
embed_configs['occupation']={"embed_dim":EMED_DIM,"num_embed":num_occupation}
embed_configs['age_group']={"embed_dim":EMED_DIM,"num_embed":num_age_group}

In [94]:
embedding_layer = EmbeddingLayer(embed_configs)
print(embedding_layer.embeddings)

ModuleDict(
  (item): Embedding(3884, 32, padding_idx=0)
  (position): Embedding(4, 32, padding_idx=0)
  (user): Embedding(6041, 32, padding_idx=0)
  (sex): Embedding(3, 32, padding_idx=0)
  (occupation): Embedding(22, 32, padding_idx=0)
  (age_group): Embedding(8, 32, padding_idx=0)
)


In [95]:
class TransformerBlock(nn.Module):
    def __init__(self, input_size, output_size, num_heads, dropout_rate):
        super(TransformerBlock, self).__init__()

        self.multihead_attention = nn.MultiheadAttention(input_size, num_heads)
        self.layer_norm1 = nn.LayerNorm(input_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(input_size, 4*input_size),
            nn.ReLU(),
            nn.Linear(4*input_size, output_size),
            nn.Dropout(dropout_rate)
        )
        self.layer_norm2 = nn.LayerNorm(output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # Multi-head Attention
        attn_output, _ = self.multihead_attention(x, x, x)
        x = self.layer_norm1(x + attn_output)

        # Feed-Forward Network
        ff_output = self.feed_forward(x)
        x = self.layer_norm2(x + ff_output)
        x = self.dropout(x)
        return x

class TransformerLayer(nn.Module):
    def __init__(self, d_model, num_heads=8, dropout_rate=0.2, num_layers=3):
        super(TransformerLayer, self).__init__()

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, d_model, num_heads, dropout_rate)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        return x

In [96]:
class MLP(nn.Module):
    def __init__(self, dropout=0.2, hidden_units=[512, 256,128]):
        super(MLP, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.layers = nn.ModuleList()
        for i in range(len(hidden_units) - 1):
            self.layers.append(nn.Linear(hidden_units[i], hidden_units[i + 1]))
            self.layers.append(nn.LeakyReLU())
            self.layers.append(nn.Dropout(p=dropout))
        self.fc = nn.Linear(hidden_units[-1],1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        logits = self.fc(x)
        output = self.sigmoid(logits)
        return output

In [ ]:
class BSTRecommender(nn.Module):    
    def __init__(self,embedding_layer,num_heads=8,transformer_num_layer=3,drop_out=0.2):
        super().__init__()
        # Embedding params
        self.seq_len = embedding_layer.seq_len
        self.totoal_dim = embedding_layer.total_dim
        self.transformer_dim = embedding_layer.transformer_dim
        
        self.drouput = drop_out
        self.num_heads = num_heads
        self.transformer_num_layer = transformer_num_layer
        
        # Embedding
        self.embedding_layer = embedding_layer
        
        # Transformer
        self.transformer_layer = TransformerLayer(d_model=self.transformer_dim,
                                            num_heads=self.num_heads,
                                            dropout_rate=self.drouput,
                                            num_layers=self.transformer_num_layer)
        
        # MLP
        self.mlp = MLP(dropout=self.drouput, hidden_units=[self.totoal_dim, 256, 64])
            
    def forward(self, batch):
        batch_size = batch['movie_sequence'].shape[0]
        # 1. 用户特征embedding
        user_features = {
            'user': batch['user_id_index'],
            'occupation': batch['occupation_index'],
            'age_group': batch['age_group_index'],
            'sex': batch['sex']
        }
        user_embeddings_dict = self.embedding_layer(user_features)
        # 2. seq embeds
        movie_embeds = self.embedding_layer.embeddings['item'](batch['movie_sequence'])
        position_ids = torch.arange(self.seq_len, device=batch['movie_sequence'].device).unsqueeze(0).expand(batch_size, -1)
        position_embeds = self.embedding_layer.embeddings['position'](position_ids)
        # concat
        sequence_embeds = torch.cat([movie_embeds, position_embeds], dim=-1)
        
        # Apply rating weights
        if 'rating_sequence' in batch:
            rating_weights = batch['rating_sequence'].unsqueeze(-1)
            sequence_embeds = sequence_embeds * rating_weights
        
        # 3. Transformer编码
        transformer_output = self.transformer_layer(sequence_embeds)
        # 4. 序列pooling (取最后一个位置)
        sequence_pooled = transformer_output[:, -1, :]  # Take last position
        # 5. 目标电影embedding
        target_movie_embed = self.embedding_layer.embeddings['item'](batch['target_movie'])
        # 6. 特征融合 - concatenate all features (FIXED)
        feature_list = []
        # Add user embeddings (including sex embedding)
        for embed in user_embeddings_dict.values():
            feature_list.append(embed)
        # Add sequence features
        feature_list.append(sequence_pooled)
        # Add target movie
        feature_list.append(target_movie_embed)
        features = torch.cat(feature_list, dim=-1)
        
        # 7. MLP预测
        output = self.mlp(features)  # Use self.mlp, not self.mlp_predictor
        
        return output.squeeze(-1)


In [98]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
model = BSTRecommender(embedding_layer=embedding_layer)
model.to(DEVICE)
model.train()

cuda


BSTRecommender(
  (embedding_layer): EmbeddingLayer(
    (embeddings): ModuleDict(
      (item): Embedding(3884, 32, padding_idx=0)
      (position): Embedding(4, 32, padding_idx=0)
      (user): Embedding(6041, 32, padding_idx=0)
      (sex): Embedding(3, 32, padding_idx=0)
      (occupation): Embedding(22, 32, padding_idx=0)
      (age_group): Embedding(8, 32, padding_idx=0)
    )
    (embedding_dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_layer): TransformerLayer(
    (transformer_blocks): ModuleList(
      (0-2): 3 x TransformerBlock(
        (multihead_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (layer_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3):

In [99]:
from torch.utils.data import Dataset, DataLoader

class BSTDataset(Dataset):
    def __init__(self, data, device):
        self.data = data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        return {
            'user_id_index': torch.tensor(row['user_id_index'], dtype=torch.long).to(self.device),
            'occupation_index': torch.tensor(row['occupation_index'], dtype=torch.long).to(self.device),
            'age_group_index': torch.tensor(row['age_group_index'], dtype=torch.long).to(self.device),
            'sex': torch.tensor(row['sex_index'], dtype=torch.long).to(self.device),
            'movie_sequence': torch.tensor(row['movie_sequence'], dtype=torch.long).to(self.device),
            'rating_sequence': torch.tensor(row['rating_sequence'], dtype=torch.float).to(self.device),
            'target_movie': torch.tensor(row['movie_sequence'][-1], dtype=torch.long).to(self.device),  # Last movie as target
            'target_rating': torch.tensor(row['rating_sequence'][-1], dtype=torch.float).to(self.device)  # Last rating as target
        }
# Create datasets
train_dataset = BSTDataset(train_data.reset_index(drop=True), DEVICE)
test_dataset = BSTDataset(test_data.reset_index(drop=True), DEVICE)

# Create dataloaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [100]:
# Training config
epochs = 10
learning_rate = 0.001
criterion = nn.MSELoss()  # or nn.L1Loss() for MAE
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [101]:
from tqdm import tqdm

# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(batch)
        
        # Calculate loss
        loss = criterion(predictions, batch['target_rating'])
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f}')

Epoch 1/10:   0%|          | 0/3275 [00:00<?, ?it/s]

Epoch 1/10: 100%|██████████| 3275/3275 [04:15<00:00, 12.83it/s, loss=7.08e-5] 


Epoch 1, Average Loss: 0.0009


Epoch 2/10: 100%|██████████| 3275/3275 [04:13<00:00, 12.92it/s, loss=0.000285]


Epoch 2, Average Loss: 0.0005


...

Epoch 8, Average Loss: 0.0004


Epoch 9/10: 100%|██████████| 3275/3275 [04:06<00:00, 13.28it/s, loss=9.65e-5] 


Epoch 9, Average Loss: 0.0002


Epoch 10/10: 100%|██████████| 3275/3275 [04:04<00:00, 13.38it/s, loss=0.000195]

Epoch 10, Average Loss: 0.0007


In [102]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# test it 
model.eval()
with torch.no_grad():
    test_loss = 0
    predictions_list = []
    targets_list = []
    
    for batch in tqdm(test_loader, desc='Testing'):
        predictions = model(batch)
        loss = criterion(predictions, batch['target_rating'])
        test_loss += loss.item()
        
        predictions_list.extend(predictions.cpu().numpy())
        targets_list.extend(batch['target_rating'].cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

# Calculate additional metrics

mae = mean_absolute_error(targets_list, predictions_list)
rmse = np.sqrt(mean_squared_error(targets_list, predictions_list))
print(f'MAE: {mae:.4f}')
print(f'RMSE: {rmse:.4f}')

# Save model
torch.save(model.state_dict(), 'bst_model.pth')
print('Model saved as bst_model.pth')

Testing: 100%|██████████| 574/574 [00:40<00:00, 14.17it/s]

Test Loss: 0.0003
MAE: 0.0035
RMSE: 0.0184
Model saved as bst_model.pth
